In [1]:
import numpy as np
from numpy import linalg
from matplotlib import pyplot as plt
import sympy as sym
from sympy.solvers import solve
from sympy import Symbol

## Rayleigh Quotient function

In [2]:
def f(A, x):
    return np.dot(x.T, np.dot(A, x))

## for the subspace find the projector to that subspace

In [3]:
def getProj(A):
    """ 
    input A is a subspace
    output is a projector that projects to this subspace
    """
    return A @ np.linalg.inv(A.T @ A) @ A.T

## from the projector find the subspace that it projects to

In [4]:
def getSubspace(P):
    """
    input P is a projector
    output is a subspace that this projector projects to
    """
    val,vec = np.linalg.eig(P)
    for i in range(3):
        if(val[i] > 1 - 1e-6):
            return vec[:,i]

## Make a random symmetric matrix

In [63]:
A = np.random.random_sample((3,3))
A = np.dot(A, A.T)
A = np.array([[181, 101, 146],
              [101, 74, 103],
              [146, 103, 146]])

## EXPECTED SOLUTIONS ARE

## Pick starting point


In [6]:
x0 = np.array([[0],[0],[1]])
P0 = getProj(x0)
getSubspace(P0)

array([0., 0., 1.])

# STEP 1
Pick an ortogonal matrix $\Theta_0 \in SO_n$ corresponding to
$$ P_0 = \Theta_0^T \begin{bmatrix} I_m & 0 \\ 0 & 0 \end{bmatrix} \Theta_0 \in Gr_{m,n}$$
and set $j=0$

In [64]:
n=3
m=1

In [65]:
H = np.random.rand(n, n)
u, s, vh = np.linalg.svd(H, full_matrices=False)
Theta = u @ vh

In [66]:
np.linalg.det(Theta)

-1.0000000000000002

In [67]:
P = Theta.T @ np.pad(np.eye(m),((0,n-m),(0,n-m))) @ Theta

In [653]:
np.linalg.matrix_rank(P)

1

## STEP 2
Compute
$$ \begin{bmatrix}
    A_{1,1} & A_{1,2} \\ 
    A_{1,2}^T & A_{2,2} 
    \end{bmatrix} = 
    \Theta_j A \Theta_{j}^T
$$

In [12]:
Res = Theta @ A @ Theta.T

In [13]:
Res

array([[66.37547439,  2.8204349 , 62.38036276],
       [ 2.8204349 ,  6.83755333,  9.4701554 ],
       [62.38036276,  9.4701554 , 68.78697228]])

$A_{1,1}$ has the shape $m \times m$

$A_{2,2}$ has the shape $(n-m) \times (n-m)$

$A_{1,2}$ has the shape $m \times (n-m)$

$Z_j$ has the shape $m \times (n-m)$

In [14]:
A11 = Res[0:m,0:m]
print(A11)
print(A11.shape)

[[66.37547439]]
(1, 1)


In [15]:
A12 = Res[:m,m:]
print(A12)
print(A12.shape)

[[ 2.8204349  62.38036276]]
(1, 2)


In [16]:
A22 = Res[m:,m:]
print(A22)
print(A22.shape)

[[ 6.83755333  9.4701554 ]
 [ 9.4701554  68.78697228]]
(2, 2)


## STEP 3 
Solve the Sylvester equation
$$ A_{1,1} Z_j - Z_j A_{2,2} = A_{1,2} $$

In [17]:
z1 = Symbol('z1')
z2 = Symbol('z2')
Z = np.array([[z1],[z2]]).T

In [18]:
Z.shape

(1, 2)

## first to solve
A11[0] * z1 - z1 * A22[0][0] + z2 * A22[0][1] == A12[0]

## second to solve
A11[1] * z2 - z1 * A22[1][0] + z2 * A22[1][1] = A12[1]

In [702]:
eq1 = sym.Eq(A11[0][0] * z1 - z1*A22[0][0] - z2* A22[1][0], A12[0][0])
eq2 = sym.Eq(A11[0][0]*z2 - z1 * A22[0][1] - z2 * A22[1][1],A12[0][1])
result = sym.solve([eq1,eq2],(z1,z2),prec=50)
print(result)

{z1: 1.64331127829724, z2: -0.00614288033593824}


In [20]:
A12.shape

(1, 2)

In [21]:
Z = np.array([[result[z1]],[result[z2]]]).T

## Step 4 
Compute 
$$ \Theta^T_{j+1} = \Theta^T_{j} 
   \begin{bmatrix} I_m & Z_j \\  
                   -Z_j^T & I_{n-m}
   \end{bmatrix}_{Q}
   \quad
   \text{and}
   \quad
   P_{j+1} = \Theta_{j+1}^T 
   \begin{bmatrix} I_m & 0 \\
                   0 & 0 
    \end{bmatrix} \Theta_{j+1}
$$

In [22]:
TEMP = np.eye(n)
TEMP[0:m,m:] = Z
TEMP[m:,:m] = -Z.T
TEMP_Q, R = np.linalg.qr(TEMP)

In [23]:
Theta = (Theta.T @ TEMP_Q).T

In [24]:
P = Theta.T @ np.pad(np.eye(m),((0,n-m),(0,n-m))) @ Theta

In [25]:
getSubspace(P)

array([-0.19419466, -0.9753497 ,  0.10479217])

In [26]:
Theta

array([[-0.19419466, -0.9753497 ,  0.10479217],
       [ 0.04270499,  0.0983189 ,  0.99423824],
       [-0.98003302,  0.19755091,  0.02255929]])

## JOIN ALL

In [108]:
# Step 2
Res = Theta @ A @ Theta.T
A11 = Res[0:m,0:m]
A12 = Res[:m,m:]
A22 = Res[m:,m:]
# STEP 3
z1 = Symbol('z1')
z2 = Symbol('z2')
Z = np.array([[z1],[z2]]).T
eq1 = sym.Eq(A11[0][0] * z1 - z1*A22[0][0] - z2* A22[1][0], A12[0][0])
eq2 = sym.Eq(A11[0][0]*z2 - z1 * A22[0][1] - z2 * A22[1][1],A12[0][1])
result = sym.solve([eq1,eq2],(z1,z2))
Z = np.array([[result[z1]],[result[z2]]]).T
# STEP 4
TEMP = np.eye(n)
TEMP[0:m,m:] = Z
TEMP[m:,:m] = -Z.T
TEMP_Q, R = np.linalg.qr(TEMP)
Theta = (Theta.T @ TEMP_Q).T
P = Theta.T @ np.pad(np.eye(m),((0,n-m),(0,n-m))) @ Theta
Theta

array([[-0.66889888, -0.48566074, -0.5627681 ],
       [ 0.02075902,  0.74456721, -0.66722465],
       [-0.74306349,  0.45798834,  0.48795833]])

## STEP 5
Set j = j+1 and goto STEP 2

In [71]:
np.linalg.eig(A)[1][:,1]

array([ 0.74148822, -0.44835462, -0.49917267])

In [72]:
np.linalg.eig(A)[1][:,2]

array([-0.05957434, -0.78501609,  0.61660411])

In [73]:
np.linalg.eig(A)[1][:,0]

array([0.66831589, 0.4274668 , 0.60879061])